In [9]:
import numpy as np
import tensorflow as tf
import pandas as pd
import h5py
import os
from pathlib import Path
from keras.preprocessing import image

Using TensorFlow backend.


# Sign Numbers

In [2]:
def load_dataset_sign():
    train_dataset = h5py.File('signs/train_signs.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File('signs/test_signs.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

In [3]:
def convert_to_one_hot(y, C):
    y = np.eye(C)[y.reshape(-1)].T
    return y

# Rock Paper Scissors

In [11]:
def load_dataset_rps():
    labels={
        'rock':0,
        'paper':1,
        'scissors':2,
    }
    p = Path("/home/tarun/Desktop/Projects Completed/ResNet/rps")
    train_set_x_orig=[]
    train_set_y_orig=[]
    direc = p.glob('*')

    for d in direc:
        fname=str(d)
        label=fname.split("/")[-1]
        if label != '.DS_Store':
            for j in d.glob('*'):
                img=image.load_img(j,target_size=(64,64))
                image_array=image.img_to_array(img)
                train_set_x_orig.append(np.array(image_array))
                train_set_y_orig.append(labels[label])
    
    train_set_x_orig=np.array(train_set_x_orig)
    train_set_y_orig=np.array(train_set_y_orig)

    test_set_x_orig=[]
    test_set_y_orig=[]
    p = Path("/home/tarun/Desktop/Projects Completed/ResNet/rps-test-set")
    direc = p.glob('*')
    for d in direc:
        fname=str(d)
        label=fname.split("/")[-1]
        if(label != '.DS_Store'):
            for j in d.glob('*'):
                img=image.load_img(j,target_size=(64,64))
                img=image.img_to_array(img)
                test_set_x_orig.append(img)
                test_set_y_orig.append(labels[label])

    test_set_x_orig=np.array(test_set_x_orig)
    test_set_y_orig=np.array(test_set_y_orig)
    
    classes = [0, 1, 2]
    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

In [13]:
X_train_orig, y_train_orig, X_test_orig, y_test_orig, classes = load_dataset_rps()

# Normalize image vectors
X_train = X_train_orig/255.
X_test = X_test_orig/255.

# Convert training and test labels to one hot matrices
y_train = convert_to_one_hot(y_train_orig, len(classes)).T
y_test = convert_to_one_hot(y_test_orig, len(classes)).T

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("number of classes = " + str(len(classes)))
print ("X_train shape: " + str(X_train.shape))
print ("y_train shape: " + str(y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("y_test shape: " + str(y_test.shape))
print(classes)

number of training examples = 2520
number of test examples = 372
number of classes = 3
X_train shape: (2520, 64, 64, 3)
y_train shape: (2520, 3)
X_test shape: (372, 64, 64, 3)
y_test shape: (372, 3)
[0, 1, 2]


In [14]:
def create_placeholders(n_h, n_w, n_c, n_y):
    '''
    Arguments:
    n_h -- height of the input image 
    n_w -- width of the input image
    n_c -- channels of the input image
    n_y -- number of classes
    
    Returns:
    X -- placeholder for the data inputs of shape (None, n_h, n_w, n_c)
    y -- placeholder for the output labels of shape (None, n_y)
    is_training -- boolean placeholder for batch normalization
    '''
    X = tf.placeholder(tf.float32, shape = (None, n_h, n_w, n_c))
    y = tf.placeholder(tf.float32, shape = (None, n_y))
    is_training = tf.placeholder_with_default(False, (), 'is_training')
    return X, y, is_training

In [15]:
def zero_padding(X, pad):
    '''
    Arguments:
    X -- input dataset
    pad -- zero padding
    
    Returns:
    X_pad -- padded input dataset
    '''
    X_pad = tf.pad(X, tf.constant([[0, 0], [pad, pad], [pad, pad], [0, 0]]), "CONSTANT")
    return X_pad

In [16]:
def initialize_parameters():
    '''
    Returns:
    parameters -- a dictionary containing tensors for weights
    '''
    parameters = {
        # Stage 1
        'b1' : {
            'w1' : tf.get_variable('w0', shape = [7, 7, 3, 64], initializer = tf.contrib.layers.xavier_initializer())
        },
        
        # Stage 2
        'b2' : {
            'w1' : tf.get_variable('w1', shape = [1, 1, 64, 64], initializer = tf.contrib.layers.xavier_initializer()),
            'w2' : tf.get_variable('w2', shape = [3, 3, 64, 64], initializer = tf.contrib.layers.xavier_initializer()),
            'w3' : tf.get_variable('w3a', shape = [1, 1, 64, 256], initializer = tf.contrib.layers.xavier_initializer()),
            'w4' : tf.get_variable('w3b', shape = [1, 1, 64, 256], initializer = tf.contrib.layers.xavier_initializer())
        },
        'b3' : {
            'w1' : tf.get_variable('w4', shape = [1, 1, 256, 64], initializer = tf.contrib.layers.xavier_initializer()),
            'w2' : tf.get_variable('w5', shape = [3, 3, 64, 64], initializer = tf.contrib.layers.xavier_initializer()),
            'w3' : tf.get_variable('w6', shape = [1, 1, 64, 256], initializer = tf.contrib.layers.xavier_initializer())
        },
        'b4' : {
            'w1' : tf.get_variable('w7', shape = [1, 1, 256, 64], initializer = tf.contrib.layers.xavier_initializer()),
            'w2' : tf.get_variable('w8', shape = [3, 3, 64, 64], initializer = tf.contrib.layers.xavier_initializer()),
            'w3' : tf.get_variable('w9', shape = [1, 1, 64, 256], initializer = tf.contrib.layers.xavier_initializer())
        },
        
        # Stage 3
        'b5' : {
            'w1' : tf.get_variable('w10', shape = [1, 1, 256, 128], initializer = tf.contrib.layers.xavier_initializer()),
            'w2' : tf.get_variable('w11', shape = [3, 3, 128, 128], initializer = tf.contrib.layers.xavier_initializer()),
            'w3' : tf.get_variable('w12a', shape = [1, 1, 128, 512], initializer = tf.contrib.layers.xavier_initializer()),
            'w4' : tf.get_variable('w12b', shape = [1, 1, 256, 512], initializer = tf.contrib.layers.xavier_initializer())
        },
        'b6' : {
            'w1' : tf.get_variable('w13', shape = [1, 1, 512, 128], initializer = tf.contrib.layers.xavier_initializer()),
            'w2' : tf.get_variable('w14', shape = [3, 3, 128, 128], initializer = tf.contrib.layers.xavier_initializer()),
            'w3' : tf.get_variable('w15', shape = [1, 1, 128, 512], initializer = tf.contrib.layers.xavier_initializer())
        },
        'b7' : {
            'w1' : tf.get_variable('w16', shape = [1, 1, 512, 128], initializer = tf.contrib.layers.xavier_initializer()),
            'w2' : tf.get_variable('w17', shape = [3, 3, 128, 128], initializer = tf.contrib.layers.xavier_initializer()),
            'w3' : tf.get_variable('w18', shape = [1, 1, 128, 512], initializer = tf.contrib.layers.xavier_initializer())
        },
        'b8' : {
            'w1' : tf.get_variable('w19', shape = [1, 1, 512, 128], initializer = tf.contrib.layers.xavier_initializer()),
            'w2' : tf.get_variable('w20', shape = [3, 3, 128, 128], initializer = tf.contrib.layers.xavier_initializer()),
            'w3' : tf.get_variable('w21', shape = [1, 1, 128, 512], initializer = tf.contrib.layers.xavier_initializer())
        },
        
        # Stage 4
        'b9' : {
            'w1' : tf.get_variable('w22', shape = [1, 1, 512, 256], initializer = tf.contrib.layers.xavier_initializer()),
            'w2' : tf.get_variable('w23', shape = [3, 3, 256, 256], initializer = tf.contrib.layers.xavier_initializer()),
            'w3' : tf.get_variable('w24a', shape = [1, 1, 256, 1024], initializer = tf.contrib.layers.xavier_initializer()),
            'w4' : tf.get_variable('w24b', shape = [1, 1, 512, 1024], initializer = tf.contrib.layers.xavier_initializer())
        },
        'b10' : {
            'w1' : tf.get_variable('w25', shape = [1, 1, 1024, 256], initializer = tf.contrib.layers.xavier_initializer()),
            'w2' : tf.get_variable('w26', shape = [3, 3, 256, 256], initializer = tf.contrib.layers.xavier_initializer()),
            'w3' : tf.get_variable('w27', shape = [1, 1, 256, 1024], initializer = tf.contrib.layers.xavier_initializer())
        },
        'b11' : {
            'w1' : tf.get_variable('w28', shape = [1, 1, 1024, 256], initializer = tf.contrib.layers.xavier_initializer()),
            'w2' : tf.get_variable('w29', shape = [3, 3, 256, 256], initializer = tf.contrib.layers.xavier_initializer()),
            'w3' : tf.get_variable('w30', shape = [1, 1, 256, 1024], initializer = tf.contrib.layers.xavier_initializer())
        },
        'b12' : {
            'w1' : tf.get_variable('w31', shape = [1, 1, 1024, 256], initializer = tf.contrib.layers.xavier_initializer()),
            'w2' : tf.get_variable('w32', shape = [3, 3, 256, 256], initializer = tf.contrib.layers.xavier_initializer()),
            'w3' : tf.get_variable('w33', shape = [1, 1, 256, 1024], initializer = tf.contrib.layers.xavier_initializer())
        },
        'b13' : {
            'w1' : tf.get_variable('w34', shape = [1, 1, 1024, 256], initializer = tf.contrib.layers.xavier_initializer()),
            'w2' : tf.get_variable('w35', shape = [3, 3, 256, 256], initializer = tf.contrib.layers.xavier_initializer()),
            'w3' : tf.get_variable('w36', shape = [1, 1, 256, 1024], initializer = tf.contrib.layers.xavier_initializer())
        },
        'b14' : {
            'w1' : tf.get_variable('w37', shape = [1, 1, 1024, 256], initializer = tf.contrib.layers.xavier_initializer()),
            'w2' : tf.get_variable('w38', shape = [3, 3, 256, 256], initializer = tf.contrib.layers.xavier_initializer()),
            'w3' : tf.get_variable('w39', shape = [1, 1, 256, 1024], initializer = tf.contrib.layers.xavier_initializer())
        },
        
        # Stage 5
        'b15' : {
            'w1' : tf.get_variable('w40', shape = [1, 1, 1024, 512], initializer = tf.contrib.layers.xavier_initializer()),
            'w2' : tf.get_variable('w41', shape = [3, 3, 512, 512], initializer = tf.contrib.layers.xavier_initializer()),
            'w3' : tf.get_variable('w42a', shape = [1, 1, 512, 2048], initializer = tf.contrib.layers.xavier_initializer()),
            'w4' : tf.get_variable('w42b', shape = [1, 1, 1024, 2048], initializer = tf.contrib.layers.xavier_initializer())
        },
        'b16' : {
            'w1' : tf.get_variable('w43', shape = [1, 1, 2048, 512], initializer = tf.contrib.layers.xavier_initializer()),
            'w2' : tf.get_variable('w44', shape = [3, 3, 512, 512], initializer = tf.contrib.layers.xavier_initializer()),
            'w3' : tf.get_variable('w45', shape = [1, 1, 512, 2048], initializer = tf.contrib.layers.xavier_initializer())
        },
        'b17' : {
            'w1' : tf.get_variable('w46', shape = [1, 1, 2048, 512], initializer = tf.contrib.layers.xavier_initializer()),
            'w2' : tf.get_variable('w47', shape = [3, 3, 512, 512], initializer = tf.contrib.layers.xavier_initializer()),
            'w3' : tf.get_variable('w48', shape = [1, 1, 512, 2048], initializer = tf.contrib.layers.xavier_initializer())
        }
    }
    return parameters

In [17]:
def identity_block(X, parameters, is_training):
    '''
    Arguments:
    X -- input dataset
    parameters -- python dictionary for weights
    is_training -- boolean tensor for batch normalization
    
    Returns:
    X -- output dataset
    '''
    X_shortcut = X
    
    X = tf.nn.conv2d(X, parameters['w1'], strides = [1,1,1,1], padding = 'VALID')
    X = tf.layers.batch_normalization(X, axis = 3, training=is_training)
    X = tf.nn.relu(X)
    
    X = tf.nn.conv2d(X, parameters['w2'], strides = [1,1,1,1], padding = 'SAME')
    X = tf.layers.batch_normalization(X, axis = 3, training=is_training)
    X = tf.nn.relu(X)
    
    X = tf.nn.conv2d(X, parameters['w3'], strides = [1,1,1,1], padding = 'VALID')
    X = tf.layers.batch_normalization(X, axis = 3, training=is_training)
    
    X = tf.add(X_shortcut, X)
    
    X = tf.nn.relu(X)
    
    return X

In [18]:
def conv_block(X, parameters, is_training, s = 2):
    '''
    Arguments:
    X -- input dataset
    parameters -- python dictionary for weights
    is_training -- boolean tensor for batch normalization
    s -- stride
    
    Returns:
    X -- output dataset
    '''
    X_shortcut = X
    
    X = tf.nn.conv2d(X, parameters['w1'], strides = [1,s,s,1], padding = 'VALID')
    X = tf.layers.batch_normalization(X, axis = 3, training=is_training)
    X = tf.nn.relu(X)
    
    X = tf.nn.conv2d(X, parameters['w2'], strides = [1,1,1,1], padding = 'SAME')
    X = tf.layers.batch_normalization(X, axis = 3, training=is_training)
    X = tf.nn.relu(X)
    
    X = tf.nn.conv2d(X, parameters['w3'], strides = [1,1,1,1], padding = 'VALID')
    X = tf.layers.batch_normalization(X, axis = 3, training=is_training)
    
    X_shortcut = tf.nn.conv2d(X_shortcut, parameters['w4'], strides = [1,s,s,1], padding = 'VALID')
    X_shortcut = tf.layers.batch_normalization(X_shortcut, axis = 3, training=is_training)
    
    X = tf.add(X_shortcut, X)
    
    X = tf.nn.relu(X)
    
    return X

In [19]:
def forward_propagation(X, parameters, is_training, n_y):
    '''
    Arguments:
    X -- input dataset placeholder of shape (m, n_h, n_w, n_c)
    parameters -- python dict of dicts for weights
    is_training -- boolean value for batch normalization
    n_y -- number of classes
    
    Returns:
    Z -- output of the last linear unit
    '''
    # Stage 1
    X = zero_padding(X, 3)
    X = tf.nn.conv2d(X, parameters['b1']['w1'], strides = [1,2,2,1], padding = 'VALID')
    X = tf.layers.batch_normalization(X, axis = 3, training=is_training)
    X = tf.nn.relu(X)
    X = tf.nn.max_pool(X, ksize = [1,3,3,1], strides = [1,2,2,1], padding = 'VALID')
    
    # Stage 2
    X = conv_block(X, parameters['b2'], is_training, s = 1)
    X = identity_block(X, parameters['b3'], is_training)
    X = identity_block(X, parameters['b4'], is_training)
    
    # Stage 3
    X = conv_block(X, parameters['b5'], is_training, s = 2)
    X = identity_block(X, parameters['b6'], is_training)
    X = identity_block(X, parameters['b7'], is_training)
    X = identity_block(X, parameters['b8'], is_training)
    
    #Stage 4
    X = conv_block(X, parameters['b9'], is_training, s = 2)
    X = identity_block(X, parameters['b10'], is_training)
    X = identity_block(X, parameters['b11'], is_training)
    X = identity_block(X, parameters['b12'], is_training)
    X = identity_block(X, parameters['b13'], is_training)
    X = identity_block(X, parameters['b14'], is_training)
    
    # Stage 5
    X = conv_block(X, parameters['b15'], is_training, s = 2)
    X = identity_block(X, parameters['b16'], is_training)
    X = identity_block(X, parameters['b17'], is_training)
    
    X = tf.nn.avg_pool(X, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'VALID')
    
    F = tf.contrib.layers.flatten(X)
    Z = tf.contrib.layers.fully_connected(F, n_y, activation_fn = None)
    
    return Z

In [20]:
def compute_cost(Z, y):
    '''
    Arguments:
    Z -- output of the last linear unit
    y -- target labels
    
    Returns:
    cost -- softmax cross entropy cost
    '''
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = Z, labels = y))
    return cost

In [21]:
def random_mini_batches(X, y, mini_batch_size = 64, seed = 0):
    '''
    Arguments:
    X -- input dataset
    Y -- target labels
    mini_batch_size -- size of mini batches
    seed -- seed value for random functions
    
    Returns:
    mini_batches -- list of random mini batches
    '''
    m = X.shape[0] 
    mini_batches = []
    np.random.seed(seed)
    
    # Step 1: Shuffle (X, Y)
    permutation = list(np.random.permutation(m))
    shuffled_X = X[permutation,:]
    shuffled_y = y[permutation,:]

    # Step 2: Partition (shuffled_X, shuffled_Y). Minus the end case.
    num_complete_mini_batches = int(m/mini_batch_size)
    for k in range(0, num_complete_mini_batches):
        mini_batch_X = shuffled_X[k * mini_batch_size : k * mini_batch_size + mini_batch_size,:]
        mini_batch_y = shuffled_y[k * mini_batch_size : k * mini_batch_size + mini_batch_size,:]
        mini_batch = (mini_batch_X, mini_batch_y)
        mini_batches.append(mini_batch)
    
    # Handling the end case (last mini-batch < mini_batch_size)
    if m % mini_batch_size != 0:
        mini_batch_X = shuffled_X[num_complete_mini_batches * mini_batch_size : m,:]
        mini_batch_y = shuffled_y[num_complete_mini_batches * mini_batch_size : m,:]
        mini_batch = (mini_batch_X, mini_batch_y)
        mini_batches.append(mini_batch)
    
    return mini_batches

In [26]:
tf.reset_default_graph()

In [27]:
m_train, n_h, n_w, n_c = X_train.shape
m_train, n_y = y_train.shape

m_test, n_h, n_w, n_c = X_test.shape
m_test, n_y = y_test.shape

px, py, is_training = create_placeholders(n_h, n_w, n_c, n_y)
parameters = initialize_parameters()

Z = forward_propagation(px, parameters, is_training, n_y)
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
cost = compute_cost(Z, py)
with tf.control_dependencies(update_ops):
    optimizer = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(cost)

predictions = tf.argmax(Z,1)
labels = tf.argmax(py,1)
results = tf.equal(predictions, labels)

init = tf.global_variables_initializer()
seed = 0

In [28]:
sess = tf.Session()
sess.run(init)

In [29]:
mini_batch_size = 32
epochs = 100
for epoch in range(epochs):
    mini_batch_cost = 0
    num_minibatches = int(m_train/mini_batch_size)
    mini_batches = random_mini_batches(X_train, y_train, mini_batch_size, seed)
    seed += 1
    for mini_batch in mini_batches:
        mini_batch_X, mini_batch_y = mini_batch
        _, c = sess.run(fetches = [optimizer, cost], feed_dict = {px : mini_batch_X, py : mini_batch_y, is_training : True})
        mini_batch_cost += c/num_minibatches

    predictions_, labels_, results_ = sess.run([predictions, labels, results], feed_dict = {px : X_train, py : y_train})
    train_score = results_.sum()/m_train
    predictions_, labels_, results_ = sess.run([predictions, labels, results], feed_dict = {px : X_test, py : y_test})
    val_score = results_.sum()/m_test
    print(f"{epoch+1}) Cost => {mini_batch_cost} | Training Accuracy => {train_score*100} | Validation Accuracy {val_score*100}")
    if mini_batch_cost < 0.0001:
        print('Training Done!!')
        break

1) Cost => 0.6404927954247871 | Training Accuracy => 33.33333333333333 | Validation Accuracy 33.33333333333333
2) Cost => 0.0780352591138394 | Training Accuracy => 33.33333333333333 | Validation Accuracy 33.33333333333333
3) Cost => 0.045933807397204866 | Training Accuracy => 37.77777777777778 | Validation Accuracy 33.33333333333333
4) Cost => 0.041731552358783106 | Training Accuracy => 88.96825396825396 | Validation Accuracy 71.7741935483871
5) Cost => 0.013540389340949682 | Training Accuracy => 99.28571428571429 | Validation Accuracy 77.15053763440861
6) Cost => 0.006960927563035022 | Training Accuracy => 99.76190476190476 | Validation Accuracy 79.83870967741935
7) Cost => 0.0025776099766447328 | Training Accuracy => 99.96031746031746 | Validation Accuracy 88.70967741935483
8) Cost => 0.0014780069967045147 | Training Accuracy => 99.72222222222223 | Validation Accuracy 70.96774193548387
9) Cost => 0.00027312657179214884 | Training Accuracy => 100.0 | Validation Accuracy 75.0
10) Cost 

In [64]:
sess.close()